In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\manob\\Documents\\Kidney_Disease_Detection\\Kidney_Disease_Detection'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [5]:
from cnnClassifier.utils.common import read_yaml, create_directories,save_json
from cnnClassifier.entity.config_entity import DataIngestionConfig


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import zipfile
import gdown
from cnnClassifier.logger import logger
from cnnClassifier.utils.common import get_size
from cnnClassifier.entity.config_entity import (DataIngestionConfig)
import shutil



class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



    def merge_original_and_augmented_datasets(self):
        """
        Merge 'Original Dataset' and 'Augmented Dataset' into a single 'Combined_Training_Data' folder.
        """

        dataset_root = os.path.join(self.config.unzip_dir, "Kidney Stone Dataset")
        dataset_paths = [
            os.path.join(dataset_root, "Original Dataset"),
            os.path.join(dataset_root, "Augmented Dataset")
        ]


        combined_training_dir = os.path.join("artifacts", "data_ingestion", "Combined_Training_Data")

        if not os.path.exists(combined_training_dir):
            os.makedirs(combined_training_dir, exist_ok=True)
            os.makedirs(os.path.join(combined_training_dir, "Stone"), exist_ok=True)
            os.makedirs(os.path.join(combined_training_dir, "Non-Stone"), exist_ok=True)

        for dataset_path in dataset_paths:
            for class_name in ["Stone", "Non-Stone"]:
                src_folder = os.path.join(dataset_path, class_name)
                dest_folder = os.path.join(combined_training_dir, class_name)

                if os.path.exists(src_folder):
                    for file in os.listdir(src_folder):
                        src_file_path = os.path.join(src_folder, file)
                        dest_file_path = os.path.join(dest_folder, file)

                        if os.path.exists(dest_file_path):
                            filename, ext = os.path.splitext(file)
                            dest_file_path = os.path.join(dest_folder, f"{filename}_copy{ext}")

                        shutil.copy(src_file_path, dest_file_path)


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.merge_original_and_augmented_datasets()
except Exception as e:
    raise e    

[2025-04-05 12:47:47,545: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-05 12:47:47,550: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-05 12:47:47,550: INFO: common: created directory at: artifacts]
[2025-04-05 12:47:47,550: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-05 12:47:47,550: INFO: 3064947772: Downloading data from https://drive.google.com/file/d/1QErXWSFqsbLv9DknfwlkPsCXedJuHpYX/view?usp=sharing into file artifacts/data_ingestion/Kidney_stone_dataset.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1QErXWSFqsbLv9DknfwlkPsCXedJuHpYX
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1QErXWSFqsbLv9DknfwlkPsCXedJuHpYX&confirm=t&uuid=cfbeecac-5423-4863-97c4-1055a9dd940d
To: c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\artifacts\data_ingestion\Kidney_stone_dataset.zip
100%|██████████| 4.05G/4.05G [02:04<00:00, 32.6MB/s] 

[2025-04-05 12:49:55,553: INFO: 3064947772: Downloaded data from https://drive.google.com/file/d/1QErXWSFqsbLv9DknfwlkPsCXedJuHpYX/view?usp=sharing into file artifacts/data_ingestion/Kidney_stone_dataset.zip]
